<a href="https://colab.research.google.com/github/VesleAnne/Diploma/blob/main/Notebooks/Telebot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Запускать на GPU (в среде выполнения выбрать T4 или любой другой серверный ускоритель).

In [ ]:
# Perform Google Colab installs
import os

if "COLAB_GPU" in os.environ:
    print("[INFO] Running in Google Colab, installing requirements.")
    !pip install -U torch -q
    !pip install tqdm -q # for progэress bars
    !pip install sentence-transformers -q # for embedding models
    !pip install accelerate -q # for quantization model loading
    !pip install bitsandbytes -q # for quantizing models (less storage space)
    !pip install pytelegrambotapi -q

[INFO] Running in Google Colab, installing requirements.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.9/242.9 kB 5.7 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import util, SentenceTransformer
import pandas as pd
import random
import numpy as np
from transformers import BitsAndBytesConfig
import textwrap

In [ ]:
import telebot

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
embedding_model = SentenceTransformer(model_name_or_path="FractalGPT/SbertDistil",
                                      device=device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.58k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/706k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/482k [00:00<?, ?B/s]

In [ ]:
from google.colab import drive

# Подключение к Google Диску
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
embeddings_df_path = "/content/drive/MyDrive/МФТИ/вкр/RAG/DATA/text_chunks_and_embeddings_df3.csv"
text_chunks_and_embedding_df = pd.read_csv(embeddings_df_path)
# Convert embedding column back to np.array (it got converted to string when it got saved to CSV)
text_chunks_and_embedding_df["embedding"] = text_chunks_and_embedding_df["embedding"].apply(lambda x: np.fromstring(x.strip("[]"), sep=" "))

# Convert texts and embedding df to list of dicts
pages_and_chunks = text_chunks_and_embedding_df.to_dict(orient="records")

# Convert embeddings to torch tensor and send to device (note: NumPy arrays are float64, torch tensors are float32 by default)
embeddings = torch.tensor(np.array(text_chunks_and_embedding_df["embedding"].tolist()), dtype=torch.float32).to(device)

In [ ]:
# Define helper function to print wrapped text
def print_wrapped(text, wrap_length=80):
    wrapped_text = textwrap.fill(text, wrap_length)
    print(wrapped_text)

In [ ]:
base_model = "IlyaGusev/saiga_llama3_8b"
tokenizer = AutoTokenizer.from_pretrained(base_model, padding_side='right')

tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# bitsandbytes parameters

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    device_map='auto',
    quantization_config=bnb_config,
    trust_remote_code=True
)

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/277 [00:00<?, ?B/s]

In [ ]:
def retrieve_relevant_resources(query: str,
                                embeddings: torch.tensor,
                                model: SentenceTransformer=embedding_model,
                                n_resources_to_return: int=5,
                                print_time: bool=True):
    """
    Embeds a query with model and returns top k scores and indices from embeddings.
    """

    # Embed the query
    query_embedding = model.encode(query,
                                   convert_to_tensor=True)

    # Get  scores on embeddings

    cos_sim_scores = util.cos_sim(a=query_embedding, b=embeddings)[0]


    scores, indices = torch.topk(input=cos_sim_scores,
                                 k=n_resources_to_return)

    return scores, indices

In [ ]:
def prompt_formatter(query: str,
                     context_items: list[dict]) -> str:

    # Join context items into one dotted paragraph
    context = "- " + "\n- ".join([item["sentence_chunk"] for item in context_items])

    # Create a base prompt with examples to help the model
    # We could also write this in a txt file and import it in if we wanted.
    base_prompt = """system
    Используй следующий контекст для ответов на вопрос.
    Дай себе возможность подумать, извлекая соответствующие отрывки из контекста перед ответом на вопрос.
    Не возвращай контекст, возвращай только ответ.
    Убедись, что твои ответы как можно более понятны и логичны.
    Не повторяйся.
    Используй следующие примеры для выбора идеального стиля ответа.

    user
    Добрый вечер! Расскажите, пожалуйста о возможности привязки системы гриндата к мессенджерам, например, к телеграму. Если я захочу создать бота анкетатора (поиск соискателей для устройств на работу), чтобы не регистрировать его самостоятельно в системе, могу я как-то подкрепить бота в телеграмм, чтобы после опроса, записать данные куда-то на ваш сервер и затем уже, после принятия решения, зарегистрировать там нового пользователя.
    assistant
    Здравствуйте. К сожалению, на данный момент чаты в других мессенджерах, кроме нашего, не поддерживаются.

    user
    Есть ли интеграция с 1С? Если да, то где можно почитать про то, как подключить 1С
    assistant
    Интеграции с 1С осуществлялись, однако документации по интеграции с 1С нет. Используйте документацию по API 1С и документацию по модулю ETL на платформе https://docs-public.gdcloud.org/platform/ru/%D0%B8%D0%BD%D1%82%D0%B5%D0%B3%D1%80%D0%B0%D1%86%D0%B8%D1%8F-%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85.html

    user
    Есть ли возможность менять задачи местами в списке драг енд дропом
    assistant
    К сожалению, менять местами задачи в списке драг енд дропом не получится.

    user
    Добрый день. Хотел бы узнать, есть ли у вас свободные вакансии без опыта работы?
    assistant
    Добрый день. Контакты отдела кадров и актуальную информацию о вакансиях можно посмотреть на сайте: https://hh.ru/employer/1993100

    user
    Позови оператора
    assistant
    Переключаю на оператора.

    user
    как сделать так, чтобы при каждом запуске ETL процесса, старые данные удалялись и оставались только актуальные, которые только что были получены?
    assistant
    В ETL на этапе сохранения нужно настроить тип связи приемника "Альтернативный ключ". В типе объекта, в который сохраняются данные, нужно задать ключ, по которому будут проверяться данные.
    \nИспользуй следующий контекст для ответа на вопрос:
    {context}
    \nРелевантные цитаты: <extract relevant passages from the context here>
    user {query}

    assistant"""

    # Update base prompt with context items and query
    base_prompt = base_prompt.format(context=context, query=query)

    return base_prompt

In [ ]:
def ask(query,
        temperature=0.4,
        max_new_tokens=512,
        format_answer_text=True,
        return_answer_only=True):
    """
    Takes a query, finds relevant resources/context and generates an answer to the query based on the relevant resources.
    """

    # Get just the scores and indices of top related results
    scores, indices = retrieve_relevant_resources(query=query,
                                                  embeddings=embeddings)

    # Create a list of context items
    context_items = [pages_and_chunks[i] for i in indices]

    # Add score to context item
    #for i, item in enumerate(context_items):
    #    item["score"] = scores[i].cpu() # return score back to CPU

    # Format the prompt with context items
    prompt = prompt_formatter(query=query,
                              context_items=context_items)

    # Tokenize the prompt
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generate an output of tokens
    outputs = model.generate(**input_ids,
                                 temperature=temperature,
                                 do_sample=True,
                                 max_new_tokens=max_new_tokens)

    # Turn the output tokens into text
    output_text = tokenizer.decode(outputs[0])

    if format_answer_text:

    # Remove the entire prompt
        output_text = output_text.split("Answer:")[1].strip()
        output_text = output_text.split("System response:")[0].strip()
        output_text = output_text.split("assistant ")[0].strip()
        output_text = output_text.split("User query:")[0].strip()
        # Replace special tokens and unnecessary help message
        output_text = output_text.replace(prompt, "").replace("<bos>", "").replace("<eos>", "").replace("<|begin_of_text|> ", "").replace("<|eot_id|>", "").replace("", "").replace("", "").replace("Sure, here is the answer to the user query:\n\n", "").replace("end  assistant", "")

        # Remove unnecessary message
        #output_text = output_text.replace("Please note that some of these examples might not be directly related to the user's question but would still provide a good example for how to respond in general.", "")

        # Remove the prompt at the beginning
        output_text = output_text.replace(prompt_formatter(query, context_items), "")
        output_text = output_text.replace(prompt, "").replace(prompt_formatter(query, context_items), "")
    # Only return the answer without the context items
    if return_answer_only:
        return output_text

    return output_text

In [ ]:
from google.colab import userdata
token = userdata.get('greendatatoken')

In [ ]:
# Создаем экземпляр бота
bot = telebot.TeleBot(token)

In [ ]:
# Function handling the user's message
def handle_user_message(query):
    try:
        # Getting response from the model
        answer = ask(query=query,
                     temperature=0.6,
                     max_new_tokens=256,
                     return_answer_only=True)

        return answer
    except Exception as e:
        print("Exception:", e)
        return "Извините, не могу ответить на ваш вопрос. Переключаю на оператора"

# Function handling the /start command
@bot.message_handler(commands=["start"])
def start(m):
    bot.send_message(m.chat.id, "Добрый день. Чем могу помочь?")

# Receiving messages from the user
@bot.message_handler(content_types=["text"])
def handle_text(message):
    try:
        # Getting user's query
        user_query = message.text

        # Handling user's message and getting response
        bot_response = handle_user_message(user_query)

        # Sending response to the user
        bot.send_message(message.chat.id, bot_response)
    except Exception as e:
        print("Exception:", e)
        bot.send_message(message.chat.id, "Извините, не могу ответить на ваш вопрос. Переключаю на оператора")

In [ ]:
# Running the bot
# Пока ячейка запущена, бот работает

bot.polling(none_stop=True, interval=0)

Ссылка на бот https://t.me/GreendataSaigabot
